In [1]:
import json
import torch

In [2]:
with open('/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/checkpoints/CAUSALDG_E3/22/metrics.json') as f:
    data = json.load(f)

In [3]:
data.keys()

dict_keys(['meanDiceAvgTargetDomains', 'meanDiceTarget', 'meanDice_targetCT', 'rawDiceTarget', 'rawDice_targetCT', 'source_meanDice', 'source_rawDice'])

In [4]:
LAST_EPOCH = -1
class_vals = torch.zeros(19,5)
for s_idx, smp in enumerate(data['rawDiceTarget']['values'][LAST_EPOCH]):
    class_vals[s_idx] = torch.tensor(smp)

In [5]:
print('mean over class', class_vals.mean(0))
print('std over class', class_vals.std(0))

mean over class tensor([0.9883, 0.7784, 0.6545, 0.5477, 0.6319])
std over class tensor([0.0107, 0.1562, 0.2573, 0.2621, 0.2266])


In [6]:
print('mean over all', class_vals.mean())
print('std over all', class_vals.std())

mean over all tensor(0.7202)
std over all tensor(0.2530)


In [7]:
import nibabel as nib

In [8]:
pred = nib.load('/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/checkpoints_test/CAUSALDG_E3/22/interm_preds/pred_targetCT_0_epoch_0.nii.gz')

In [9]:
torch.as_tensor(pred.get_fdata()).shape
# vs. (192, 160, 192)

torch.Size([192, 192, 48])

In [10]:
src_large = nib.load('/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/sourceMR/processed/image_0.nii.gz')
src = nib.load('/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/sourceMR/processed_large/image_0.nii.gz')

In [11]:
src.shape

(192, 160, 192)

In [12]:
src_large.shape

(192, 192, 48)

In [17]:
from pathlib import Path
import numpy as np
import nibabel as nib
# src_dir = '/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/sourceMR/processed_large'
# tgt_dir = '/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/sourceMR/processed_large_192_192_160'

src_dir = '/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large'
tgt_dir = '/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large_192_192_160'

# src_dir = '/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E4/sourceCT/original'
# tgt_dir = '/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E4/sourceCT/processed_large_192_192_160'

# src_dir = '/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E4/targetMR/original'
# tgt_dir = '/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E4/targetMR/processed_large_192_192_160'


for _fl in Path(src_dir).glob('*.nii.gz'):
    fl = nib.load(_fl)
    fl_160_mid = fl.get_fdata()
    fl_160_last = np.transpose(fl_160_mid, (0, -1,-2))
    if 'label' in _fl.name:
        fl_160_last[fl_160_last > 4] = 0 # drop off the last two classes
    if 'targetCT' in str(_fl):
        fl_160_last[fl_160_last == 3] = -1 # switch classes
        fl_160_last[fl_160_last == 4] = 3 # switch classes
        fl_160_last[fl_160_last == -1] = 4 # switch classes
        
    fl_160_last_nib = nib.Nifti1Image(fl_160_last, fl.affine, fl.header)
    target_file = Path(tgt_dir) / _fl.name
    if not target_file.parent.exists():
        target_file.parent.mkdir(parents=True)
    print(target_file, fl_160_last.shape)
    fl_160_last_nib.to_filename(target_file)

/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large_192_192_160/image_0.nii.gz (192, 192, 160)
/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large_192_192_160/image_1.nii.gz (192, 192, 160)
/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large_192_192_160/image_2.nii.gz (192, 192, 160)
/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large_192_192_160/image_3.nii.gz (192, 192, 160)
/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large_192_192_160/image_4.nii.gz (192, 192, 160)
/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large_192_192_160/image_5.nii.gz (192, 192, 160)
/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large_192_192_160/image_6.nii.gz (192, 192, 160)
/shared/CIMDG/Causality-Medical-Image-Domain-Gen

In [54]:
import re
import nibabel as nib
from pathlib import Path
import numpy as np
# exp_name = 'CAUSALDG_E3/22'
exp_name = 'CAUSALDG_E4/8'
fls = list(Path(f'/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/checkpoints_test/{exp_name}/interm_preds').glob('*.nii.gz'))
new_target = f'/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/checkpoints_test/{exp_name}/interm_preds_resampled'
for fl in fls:
    filename = fl.name

    found = re.search('([CTMR]{2})_(\d+)', filename)
    no = found.group(2)
    mod = found.group(1)
    new_name = f"Visceral_{mod}_{int(no):03d}.nii.gz"

    ni = nib.load(fl)
    d = ni.get_fdata()
    t = torch.as_tensor(d)
    r = torch.nn.functional.interpolate(t[None, None], size=(192, 192, 160), mode='nearest')[0, 0]
    r = r.permute(0,2,1)
    new_affine = ni.affine.copy()
    new_affine[1,:],new_affine[2,:] = new_affine.copy()[2,:],new_affine.copy()[1,:]
    new_affine[:,1],new_affine[:,2] = new_affine.copy()[:,2],new_affine.copy()[:,1]
    new_affine[:,1] *= 48/160

    print(new_affine)
    ninew = nib.Nifti1Image(r.numpy(), np.eye(4))
    nt = Path(new_target) 
    nt.mkdir(parents=True, exist_ok=True)
    ninew.to_filename(nt / new_name)
    print(new_name)


[[  2.0027473   -0.           0.         -10.01373672]
 [  0.           2.3971962    0.          44.94429779]
 [  0.          -0.           1.66666663 -27.6541729 ]
 [  0.           0.           0.           1.        ]]
Visceral_MR_006.nii.gz
[[  2.00151563  -0.           0.         -10.0075779 ]
 [  0.           2.39959588   0.          44.96147156]
 [  0.          -0.           1.66666663 -26.55460739]
 [  0.           0.           0.           1.        ]]
Visceral_MR_007.nii.gz
[[  2.0027473   -0.           0.         -10.01373672]
 [  0.           2.3971962    0.          44.93662643]
 [  0.          -0.           1.66666663 -27.21473885]
 [  0.           0.           0.           1.        ]]
Visceral_MR_008.nii.gz
[[  2.0027473   -0.           0.         -10.01373672]
 [  0.           2.3971962    0.          44.88993835]
 [  0.          -0.           1.66666663 -25.97114944]
 [  0.           0.           0.           1.        ]]
Visceral_MR_009.nii.gz
[[  2.0027473   -0.     

In [41]:
160/48

3.3333333333333335

In [ ]:
new_affine

In [14]:
import torch
from pathlib import Path
import nibabel as nib

In [15]:
p = Path("/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/targetCT/processed_large_192_192_160")

In [20]:
labels = p.glob("label*")
for l in labels:
    # print(l)
    print(torch.as_tensor(nib.load(l).get_fdata()).unique())

tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)


In [21]:

p = Path("/shared/CIMDG/Causality-Medical-Image-Domain-Generalization/data/E3/sourceMR/processed_large_192_192_160")
labels = p.glob("label*")
for l in labels:
    # print(l)
    print(torch.as_tensor(nib.load(l).get_fdata()).unique())

tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
tensor([0.0000, 1.0000, 2.0000, 3.0000, 4.0000], dtype=torch.float64)
